In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from random import randint
from pickle import load

In [2]:
url = "http://www.popvortex.com/music/charts/top-100-songs.php"

In [3]:
response = requests.get(url)

In [4]:
response.status_code

200

In [5]:
# 4.1. parse html (create the 'soup')
soup = BeautifulSoup(response.content, "html.parser")

In [6]:
# Getting the Position

In [7]:
position = []
for i in soup.select("p.chart-position"):
    position.append(i.get_text())


In [8]:
# Extracting Titles

In [9]:
title = []
# for t in soup.select(".title-artist"):
#     title.append(t.cite.get_text())
    
for t in soup.select("p.title-artist cite.title"):
    title.append(t.get_text())


In [10]:
# Extracting Artist Name

In [11]:
artist_name = []
for t2 in soup.select("p.title-artist em.artist"):
    artist_name.append(t2.get_text())
    

In [12]:
#soup.select("p.title-artist")

In [13]:
repertoire = pd.DataFrame({"position":position,
                           "song_title":title,
                           "Artist_Name":artist_name                           
                          })

# Part 2



In [14]:
repertoire


,position,song_title,Artist_Name
0,1,TEXAS HOLD 'EM,Beyoncé
1,2,Lose Control,Teddy Swims
2,3,Beautiful Things,Benson Boone
3,4,TEXAS HOLD 'EM,Beyoncé
4,5,dont let me go,mgk
...,...,...,...
95,96,Party In the U.S.A.,Miley Cyrus
96,97,Strangers,Kenya Grace
97,98,Is It Over Now? (Taylor's Version) [From The V...,Taylor Swift
98,99,Don't Stop Praying,Matthew West


In [15]:
#Coverting the song_title into lower case -- in order to compare it easily


In [16]:
repertoire["song_title"] = repertoire["song_title"].str.lower()

In [17]:
repertoire["song_title"] = repertoire["song_title"].str.strip()

In [18]:
user_song = input("Please give your Song Title: ")


Please give your Song Title: dont let me go


In [19]:
if len(user_song) > 3:
    user_song = user_song.lower().strip()
else:
    user_song = input("Please give your Song Title: ")
        

In [20]:
user_song 

'dont let me go'

In [21]:
result = repertoire["song_title"].apply(lambda x: user_song in x)
if (result == False).all() != True:
     print(repertoire['song_title'][randint(0,99)])
else:
     print("Sorry we dont have any recomandation")

standing next to you (holiday remix)


In [22]:
result.value_counts() # it check weather the user song is in the list or not


False    99
True      1
Name: song_title, dtype: int64

# Part 3

In [23]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Set up your Spotify credentials
secrets_file = open("secrets.txt","r")
#string = secrets_file.read()


In [24]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        #print(line.split(':'))
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

In [25]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

In [26]:
song_name = user_song 

In [27]:
#Searching user's song on spotify

In [28]:
results = sp.search(q=song_name, limit=1)

In [29]:
track = results['tracks']['items'][0]

In [30]:
track_uri = track["uri"]

In [31]:
track_uri 

'spotify:track:2qvGI2uUNgNheWnfxQr25W'

In [32]:
# Retrieve audio features
audio_features_user_song = sp.audio_features(track_uri)
print(audio_features_user_song)

[{'danceability': 0.502, 'energy': 0.458, 'key': 3, 'loudness': -11.385, 'mode': 1, 'speechiness': 0.349, 'acousticness': 0.607, 'instrumentalness': 0.000801, 'liveness': 0.0903, 'valence': 0.172, 'tempo': 93.301, 'type': 'audio_features', 'id': '2qvGI2uUNgNheWnfxQr25W', 'uri': 'spotify:track:2qvGI2uUNgNheWnfxQr25W', 'track_href': 'https://api.spotify.com/v1/tracks/2qvGI2uUNgNheWnfxQr25W', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2qvGI2uUNgNheWnfxQr25W', 'duration_ms': 173858, 'time_signature': 3}]


In [ ]:
#Adding popularity to audio feautre

In [33]:
# Extract the track popularity
popularity = track['popularity']
print("Popularity:", popularity)

Popularity: 0


In [34]:
import pandas as pd
from pandas import json_normalize
audio_features_user_song = json_normalize(audio_features_user_song)

In [35]:
all_audio_feautres = audio_features_user_song

In [36]:
audio_features_user_song["popularity"] = track['popularity'] 

In [37]:
audio_features_user_song.drop(["type","id","uri","track_href","analysis_url"],axis =1,inplace =True)

In [38]:
audio_features_user_song

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,popularity
0,0.502,0.458,3,-11.385,1,0.349,0.607,0.000801,0.0903,0.172,93.301,173858,3,0


In [39]:
first_column = audio_features_user_song.pop("popularity")
audio_features_user_song.insert(0,popularity,first_column)

In [40]:
audio_features_user_song

,0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0,0.502,0.458,3,-11.385,1,0.349,0.607,0.000801,0.0903,0.172,93.301,173858,3


In [41]:
audio_features_user_song = audio_features_user_song.rename(columns = {0:"popularity"})

In [42]:
audio_features_user_song

,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0,0.502,0.458,3,-11.385,1,0.349,0.607,0.000801,0.0903,0.172,93.301,173858,3


In [43]:
pscaler = load(open("scaler_new.p",'rb'))

In [ ]:
#Scaling of user song

In [ ]:
#X.columns = X.columns.astype(str)

In [44]:
X_scaled_user_song = pscaler.transform(audio_features_user_song)

In [45]:
X_scaled_user_song

array([[-1.0563414 , -0.16751876, -0.86550852, -0.62560425, -0.86307946,
         0.74983011,  3.30209234,  1.21021061, -0.50756998, -0.67057034,
        -1.14242859, -0.97982241, -0.67225194, -2.37045087]])

In [ ]:
# assigning the above song into cluster

In [46]:
kmodel = load(open("kmeans.pkl","rb"))

In [47]:
user_Song_cluster = kmodel.predict(audio_features_user_song)

C:\Users\Denish\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(


In [ ]:
# Assigning the random song from cluster 

In [48]:
song_data = pd.read_csv("Fianl_data_K_cluster.csv")

In [49]:
song_data.drop(["Unnamed: 0"],axis =1,inplace =True)

In [50]:
duplicate_rows = song_data.duplicated()
duplicate_rows.value_counts()

False    15875
True       602
dtype: int64

In [51]:
song_data.head()

,song_name,name,Num_cluster
0,Pretty Pimpin,Kurt Vile,9
1,I Need My Girl,The National,2
2,Strange,Galaxie 500,12
3,Todeswalzer,Windir,0
4,Blinding Lights,The Weeknd,12


In [52]:
for i in range(0,song_data["Num_cluster"].nunique()):
    if i == user_Song_cluster:
        new_df = song_data[song_data["Num_cluster"] == i]
        new_df.reset_index(drop=True, inplace=True)
        recomende_song = new_df['song_name'][randint(new_df.index.min(),new_df.index.max())]
print(recomende_song)        

Holy Mountain


In [53]:
#final check for weather the recomended song fall in the expected cluster or not
new_df[new_df['song_name'] == recomende_song]

,song_name,name,Num_cluster
144,Holy Mountain,Sleep,5
